#### Helper Methode zum Rendern des Zustandes im Jupyter Notebook

In [ ]:
from IPython import display
import matplotlib.pyplot as plt
%matplotlib inline

def show_state(env, step=0, info=""):
    plt.imshow(env.render(mode='rgb_array'))
    plt.title("%s | Step: %d %s" % (env.spec.id,step,info))
    plt.axis('off')

    display.clear_output(wait=True)
    display.display(plt.gcf())

# CartPole

## Aufgabe 2

Löse das CartPole-v0 (https://gym.openai.com/envs/CartPole-v0/) Environment mittels des bereits bekannten Q-Learning. Beachte hierbei, dass es sich um einen continuous state-space handelt und dieser für den tabellelarischen Ansatz zuerst in einen discrete Space transformiert werden muss.

**Hinweis:** Für die Lösung müssen nicht alle zur Verfügung stehenden Eingabefeatures genutzt werden. 

### Übersicht über die vom Environment bereitgestellten Features und deren Wertebereiche

- x (Wagenposition) ∈ [-4.8, 4.8]
- x’ (Wagengeschwindigkeit) ∈ [-3.4 10^38, 3.4 10^38]
- theta (Neigungswinkel) ∈ [-0.42, 0.42]
- theta’ (Winkelgeschwindigkeit) ∈ [-3.4 10^38, 3.4 10^38]

In [ ]:
%run ../setup.ipynb

In [ ]:
import math

def discretize(env, buckets, obs):
    upper_bounds = [env.observation_space.high[0], 0.5, env.observation_space.high[2], math.radians(50)]
    lower_bounds = [env.observation_space.low[0], -0.5, env.observation_space.low[2], -math.radians(50)]
    ratios = [(obs[i] + abs(lower_bounds[i])) / (upper_bounds[i] - lower_bounds[i]) for i in range(len(obs))]
    new_obs = [int(round((buckets[i] - 1) * ratios[i])) for i in range(len(obs))]
    new_obs = [min(buckets[i] - 1, max(0, new_obs[i])) for i in range(len(obs))]
    return tuple(new_obs)

In [ ]:
from lib.statistics import plot

n_episodes = 1000
max_steps = 200

def interact_with_environment(env, agent, buckets, verbose=False):
    statistics = []
    
    if verbose:
        print('Startposition:')
        env.render()
    
    for episode in range(n_episodes):
        done = False
        total_reward = 0
        state = env.reset()
        state = discretize(env, buckets, state) # transform state

        for t in range(max_steps):
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            next_state = discretize(env, buckets, next_state)
            
            agent.train((state, action, next_state, reward, done))
            
            state = next_state
            total_reward += reward
            
            if done:
                break
        
        if verbose:
            print(f'episode: {episode}/{n_episodes}, score: {total_reward}, steps:{t}, e:{agent.epsilon:.2f}')
            env.render()
            
        statistics.append({
            'episode': episode,
            'score': total_reward,
            'steps': t
        })
        
    return statistics

### 2.1

Laut Schätzungen von Astonomen besteht das Universum aus etwa $10^{80}$ Atome. Ausgeschrieben sind das  100000000000000000000000000000000000000000000000000000000000000000000000000000000 Atome. Wie viele Zustände können in dem CartPole Environment auftreten, wenn die oben angegebenen Wertebereiche betrachtet werden? Welches Problem ergibt sich daraus für die Reinforcement Learning Aufgabe?

### 2.2
Implementiere in **agent.py** einen Agenten, der in der Lage ist das CartPole Environment zu lösen. Definiere dafür eine angemessene Diskretisierung der state-spaces.

In [ ]:
import gym
env = gym.make('CartPole-v0')

In [ ]:
from agent_solution import AdvancedQLearning

action_size = env.action_space.n

# Hyperparams
gamma = 0.75
epsilon = 0.001
epsilon_min = 0.0001
alpha = 1.0
alpha_min = 0.001
buckets = (1, 1, 1, 1) # TODO: Define appropriate bucket sizes

agent = AdvancedQLearning(action_size=action_size, buckets=buckets, gamma=gamma, 
                          epsilon=epsilon, epsilon_min=epsilon_min, 
                          alpha=alpha, alpha_min=alpha_min)

statistics = interact_with_environment(env, agent, buckets, verbose=False)
plot(statistics)

### 2.3
Benutze GridSearch, um die Hyperparameter zu optimieren. Der `score` sollte bei `200` konvergieren.

In [ ]:
from lib.hyperparameter_optimization import GridSearch
from functools import partial

grid_search = GridSearch(
    grid_params = {
        'gamma': [], # TODO
        'epsilon': [], # TODO
        'alpha': [], # TODO
        'alpha_min': [], # TODO
        'epsilon_min': [], # TODO
        'buckets': [] # TODO
    },
    fixed_params = {
        'action_size': env.action_space.n
    },
    construct_env = partial(gym.make, 'CartPole-v0'),
    construct_agent = AdvancedQLearning,
    evaluation_func = interact_with_environment,
    grid_params_for_evaluation_func = ['buckets'],
    score_parameter = 'score'
)
grid_search.run()